In [1]:
import numpy as np
from numpy.linalg import inv
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier, MLPRegressor

# Load datasets
path_final = '/Users/nealshankar/Desktop/Assignment1/datasets-part1/tictac_final.txt'
path_multi = '/Users/nealshankar/Desktop/Assignment1/datasets-part1/tictac_multi.txt'
path_single = '/Users/nealshankar/Desktop/Assignment1/datasets-part1/tictac_single.txt'

A_final = np.loadtxt(path_final)
X_final = A_final[:, :-1]
y_final = A_final[:, -1]

A_multi = np.loadtxt(path_multi)
X_multi = A_multi[:, :-1]
y_multi = A_multi[:, -1]

A_single = np.loadtxt(path_single)
X_single = A_single[:, :-1]
y_single = A_single[:, -1]

# StandardScaler to normalize input features
scaler = StandardScaler()
X_final_scaled = scaler.fit_transform(X_final)
X_multi_scaled = scaler.fit_transform(X_multi)
X_single_scaled = scaler.fit_transform(X_single)

# Convert continuous y to binary labels for classifiers
y_final_binary = np.where(y_final > 0, 1, 0)
y_multi_binary = np.where(y_multi > 0, 1, 0)
y_single_binary = np.where(y_single > 0, 1, 0)

# Define function for evaluating classifiers with normalized confusion matrix
def evaluate_classifier(model, X, y, model_name):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    y_pred = cross_val_predict(model, X, y, cv=kfold)
    accuracy = accuracy_score(y, y_pred)
    conf_matrix = confusion_matrix(y, y_pred, normalize='true')
    print(f"{model_name} - Accuracy: {accuracy}")
    print("Normalized Confusion Matrix:")
    print(conf_matrix)
    print("-------")

# Define function for evaluating regressors with normalized confusion matrix
def evaluate_regressor(model, X, y, model_name):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    y_pred = cross_val_predict(model, X, y, cv=kfold)
    accuracy = accuracy_score(y, y_pred.round())
    conf_matrix = confusion_matrix(y, y_pred.round(), normalize='true')
    
    print(f"{model_name}")
    print(f"Accuracy: {accuracy}")
    print(f"CV Score: {-scores.mean()}")
    print(f"RMSE: {rmse}")
    print("Normalized Confusion Matrix:")
    print(conf_matrix)
    print("-------")

# Classifiers
classifiers = {
    "KNN Classifier": KNeighborsClassifier(n_neighbors=5),
    "SVC": SVC(C=1.0, kernel='linear', max_iter=10000),
    "MLP Classifier": MLPClassifier(max_iter=2000, learning_rate='adaptive'),
}

# Evaluate classifiers on scaled data
print("Evaluating classifiers on tictac_final:")
for name, model in classifiers.items():
    evaluate_classifier(model, X_final_scaled, y_final_binary, name)

print("\nEvaluating classifiers on tictac_single:")
for name, model in classifiers.items():
    evaluate_classifier(model, X_single_scaled, y_single_binary, name)

# Custom Linear Regression using Normal Equations
class LinearRegressionNE:
    def __init__(self):
        self.theta_ = None

    def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Add bias term
        self.theta_ = inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X] 
        return X_b.dot(self.theta_)

    def get_params(self, deep=True):
        return {}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self


# Regressors with LinearRegressionNE for tictac_multi
regressors = {
    "KNN Regressor": KNeighborsRegressor(n_neighbors=5),
    "Linear Regression": LinearRegressionNE(), 
    "MLP Regressor": MLPRegressor(max_iter=2000, learning_rate='adaptive'),
}

# Evaluate regressors on scaled tictac_multi
print("\nEvaluating regressors on tictac_multi:")
for name, model in regressors.items():
    evaluate_regressor(model, X_multi_scaled, y_multi, name)

Evaluating classifiers on tictac_final:
KNN Classifier - Accuracy: 0.9958246346555324
Normalized Confusion Matrix:
[[0.98795181 0.01204819]
 [0.         1.        ]]
-------
SVC - Accuracy: 0.9832985386221295
Normalized Confusion Matrix:
[[0.95180723 0.04819277]
 [0.         1.        ]]
-------
MLP Classifier - Accuracy: 0.9843423799582464
Normalized Confusion Matrix:
[[0.95481928 0.04518072]
 [0.         1.        ]]
-------

Evaluating classifiers on tictac_single:
KNN Classifier - Accuracy: 0.9186383758204855
Normalized Confusion Matrix:
[[0.77941176 0.22058824]
 [0.03769801 0.96230199]]
-------
SVC - Accuracy: 0.7612578232330942
Normalized Confusion Matrix:
[[0. 1.]
 [0. 1.]]
-------
MLP Classifier - Accuracy: 0.9717600366356282
Normalized Confusion Matrix:
[[0.94053708 0.05946292]
 [0.01844796 0.98155204]]
-------

Evaluating regressors on tictac_multi:
KNN Regressor
Accuracy: 0.857578995573195
CV Score: 0.09852045114445591
RMSE: 0.3138796762207708
Normalized Confusion Matrix:
[[

In [2]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

path_single = '/Users/nealshankar/Desktop/Assignment1/datasets-part1/tictac_single.txt'

# Load the dataset for single moves
A_single = np.loadtxt(path_single)
X_single = A_single[:, :-1]  # Features: the board states
y_single = A_single[:, -1]   # Target: the best move for player O

# Scale the features
scaler = StandardScaler()
X_single_scaled = scaler.fit_transform(X_single)

# Train the MLP classifier
mlp_model = MLPClassifier(max_iter=2000, learning_rate='adaptive')
mlp_model.fit(X_single_scaled, y_single)

class TicTacToe:
    def __init__(self, mlp_model):
        self.board = np.zeros((3, 3), dtype=int)
        self.game_over = False
        self.winner = None
        self.mlp_model = mlp_model

    def print_board(self):
        symbols = {0: " ", 1: "X", -1: "O"}
        print("\nCurrent board:")
        for row in self.board:
            print("|" + "|".join(symbols[cell] for cell in row) + "|")

    def check_winner(self):
        lines = [
            self.board[0, :], self.board[1, :], self.board[2, :],
            self.board[:, 0], self.board[:, 1], self.board[:, 2],
            self.board.diagonal(), np.fliplr(self.board).diagonal()
        ]
        for line in lines:
            if np.all(line == 1):
                self.game_over = True
                self.winner = 'X'
                return
            if np.all(line == -1):
                self.game_over = True
                self.winner = 'O'
                return
        if not np.any(self.board == 0):
            self.game_over = True
            self.winner = 'Draw'

    def human_move(self, x, y):
        if 0 <= x < 3 and 0 <= y < 3:  # Check if the move is within the board bounds
            if self.board[x, y] == 0:
                self.board[x, y] = 1
                self.check_winner()
            else:
                print("Invalid move. The cell is already occupied. Try again.")
                return False
        else:
            print("Invalid move. Please enter a row and column within the range 0-2.")
            return False
        return True

    def ai_move(self):
        # Flatten the board and prepare it for prediction
        board_state = self.board.flatten().reshape(1, -1)
        board_state_scaled = scaler.transform(board_state)  # Scale the board state
        # Predict the best move for O
        move = int(self.mlp_model.predict(board_state_scaled))
        if self.board.flatten()[move] == 0:
            self.board = self.board.flatten()
            self.board[move] = -1
            self.board = self.board.reshape(3, 3)
            self.check_winner()
        else:
            print("Invalid move.")

    def play_game(self):
        self.print_board()
        while not self.game_over:
            valid_move = False
            while not valid_move:
                try:
                    x, y = map(int, input("Enter your move (row col): ").split())
                    valid_move = self.human_move(x, y)
                except ValueError:
                    print("Please enter row and column as two numbers separated by a space.")
            if valid_move:
                self.print_board()
                if self.game_over:
                    break
                # AI move
                print("AI is making a move...")
                self.ai_move()
                self.print_board()

        print(f"Game Over. Winner: {self.winner}")

if __name__ == "__main__":
    game = TicTacToe(mlp_model)
    game.play_game()


Current board:
| | | |
| | | |
| | | |
Enter your move (row col): 1 1

Current board:
| | | |
| |X| |
| | | |
AI is making a move...

Current board:
|O| | |
| |X| |
| | | |
Enter your move (row col): 1 1
Invalid move. The cell is already occupied. Try again.
Enter your move (row col): 2 0

Current board:
|O| | |
| |X| |
|X| | |
AI is making a move...

Current board:
|O| |O|
| |X| |
|X| | |
Enter your move (row col): 20 90
Invalid move. Please enter a row and column within the range 0-2.
Enter your move (row col): 0 1

Current board:
|O|X|O|
| |X| |
|X| | |
AI is making a move...

Current board:
|O|X|O|
| |X| |
|X|O| |
Enter your move (row col): 2 2

Current board:
|O|X|O|
| |X| |
|X|O|X|
AI is making a move...

Current board:
|O|X|O|
|O|X| |
|X|O|X|
Enter your move (row col): 1 1
Invalid move. The cell is already occupied. Try again.
Enter your move (row col): 1 2

Current board:
|O|X|O|
|O|X|X|
|X|O|X|
Game Over. Winner: Draw
